In [32]:
import os
import time
import pickle
import pandas as pd
import numpy as np
from src.utils import read_eeg_signal_from_file

labels = []
data = []
baseline = []

data_path = 'data/data_preprocessed_python'

for filename in os.listdir('data/data_preprocessed_python'):
    file_path = os.path.join(data_path, filename)
    trial = read_eeg_signal_from_file(file_path)
    labels.append(trial['labels'])
    data.append(trial['data'][:, :32, :])  # leave only eeg channels

labels = np.array(labels)
data = np.array(data)

In [33]:
data.shape

(32, 40, 32, 8064)

In [34]:
channels_id = [
    0, 1, 2, 3, 5, 10, 12,
    16, 18, 19, 20, 24, 29, 30
]
sample_rate = 128

In [35]:
example_series = pd.Series(data[0][0][0][18 * sample_rate: 24*sample_rate])

In [36]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=example_series.index, y=example_series))
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Signal')

In [37]:
import plotly.io as pio

pio.templates.default = "plotly_white"

import os

if not os.path.exists("images"):
    os.mkdir("images")

In [38]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=example_series.index / 128, y=example_series))
fig.update_xaxes(title_text='Time (s)')
fig.update_yaxes(title_text='Signal')
fig.update_layout(
    width=900,
    height=400,
    margin=dict(
        l=75,
        r=75,
        b=50,
        t=50,
        pad=4
    ),
)

fig.write_image("images/fig1.png")

In [39]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=example_series[sample_rate: sample_rate*2].index, y=example_series[sample_rate: sample_rate*2]))
fig.update_xaxes(title_text='t')
fig.update_layout(
    width=400,
    height=400,
    margin=dict(
        l=25,
        r=25,
        b=25,
        t=25,
        pad=4
    ),
)

fig.write_image("images/fig2.png")

In [40]:
from src.fourier import psd_from_eeg

psd_config = dict(
    selected_channels = range(1),
    freq_bands={'Theta': [4, 7],
                'Low Alpha': [7, 10],
                'High Alpha': [10, 14],
                'Low Beta': [14, 20],
                'High Beta': [20, 28],
                'Gamma': [28, 45]},
    window_size=128,
    step_size=128,
    sample_rate=128,
)

psd_data = psd_from_eeg(np.array([example_series[sample_rate: sample_rate*2]]).T, **psd_config)
psd_data

array([[0.95244654, 0.98121274, 0.97113923, 0.52772012, 0.49248704,
        0.29486162]])

In [41]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(y=psd_data[0]))
fig.update_xaxes(title_text='Band')
fig.update_yaxes(title_text='Intensity')
